# Image quality comparison for NFL Helmet comp

This notebook provides simple tools to analyze different image formats when converting video files to frames. If you think this is useful, please throw an upvote =)

Considering that a typical image size of those images can vary from:
* 0.08 MB for JPG
* 1.42 MB for PNG
* 2.76 MB for MBP

I think it is essential to see if it is worth the trouble. This is especially true as PNG/BMP files can easily surpass the 19GB maximum disk space of Kaggle.

In my opinion, the video encoder already compresses the image enough that further converting it to JPG won't degrade the image too much. I am happy to hear your feedback!

That is it for now! more coming soon! =)



In [ ]:
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import os

In [ ]:
def get_frame_from_video(video, frame, file_format):
    f_name = f'frame.{file_format}'
    frame = frame - 1
    !ffmpeg \
        -hide_banner \
        -loglevel fatal \
        -nostats \
        -i $video -vf "select=eq(n\,$frame)" -vframes 1 $f_name
    print(f'Loading {file_format} file of size: {(os.path.getsize(f_name)/1e6):.2f} MB')
    img = Image.open(f_name)
    os.remove(f_name)
    return img

def zoom_images(bbox, imgs, labels, size = None):
    
    if size is None:
        h, w = bbox[3] - bbox[1], bbox[2] - bbox[0]
    else:
        h, w = size, size
    img_out = Image.new('RGB', (len(imgs) * w, h))
    for i, img in enumerate(imgs):
        img = img.crop(bbox)
        if size is not None:
            font = ImageFont.truetype("../input/font-dataset/FontScripts/Arabic.ttf", size//10)
            img = img.resize((size,size), Image.NEAREST)
        else: font = ImageFont.truetype("../input/font-dataset/FontScripts/Arabic.ttf", 10)
        draw = ImageDraw.Draw(img)
        draw.text((10, 10),labels[i],(255,255,255),font=font)
        img_out.paste(im=img, box=(i*w, 0))
    return img_out

class compare_frames():
    def __init__(self, video, frame, ext = ['jpg', 'png', 'bmp']):
        self.ext = ext
        imgs = []
        for e in ext:
            imgs.append(get_frame_from_video(video, frame, e))
        self.imgs = imgs
    def show(self, xc, yc, size, resize = 600):
        bbox = (xc - size//2, yc - size//2, xc + size//2, yc + size//2)
        return zoom_images(bbox, self.imgs, self.ext, resize)

In [ ]:
comp = compare_frames('../input/nfl-health-and-safety-helmet-assignment/train/57583_000082_Endzone.mp4', 1)

In [ ]:
comp.show(280, 380, 150)

In [ ]:
comp = compare_frames('../input/nfl-health-and-safety-helmet-assignment/train/57676_003572_Sideline.mp4', 50)

In [ ]:
comp.show(220, 240, 100)

In [ ]:
comp = compare_frames('../input/nfl-health-and-safety-helmet-assignment/train/57594_000923_Endzone.mp4', 150)

In [ ]:
comp.show(200, 375, 200)